In [6]:
cd /Users/alessandropreviero/Desktop/vaccine-allocation/src

/Users/alessandropreviero/Desktop/vaccine-allocation/src


In [7]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.graph_objects as go
import plotly.express as px
import us
import os

In [8]:
import main
from models.prescriptive_delphi_model import *
from pipeline.data_loading import *
from pipeline.data_processing import *
from pipeline.constants import *
from pipeline.scenario import *

In [24]:
scenario_params_grid = [
        dict(
            **dates,
            vaccine_effectiveness=vaccine_effectiveness,
            daily_vaccine_budget=daily_vaccine_budget,
            min_allocation_factor=min_allocation_factor
        )
        for dates in DATES_GRID
        for vaccine_effectiveness in VACCINE_EFFECTIVENESS_GRID
        for daily_vaccine_budget in DAILY_VACCINE_BUDGET_GRID
        for min_allocation_factor in MIN_ALLOCATION_FACTOR_GRID
    ]

scenario_params = scenario_params_grid[0]

start_date = scenario_params["start_date"]
end_date = scenario_params["start_date"]
mortality_rate_path = f"{MORTALITY_RATES_PATH}2021-base.npy"
reload_mortality_rate = os.path.isfile(mortality_rate_path)

model = Scenario(**scenario_params).load_model(
    mortality_rate_path=mortality_rate_path
)

fake_vaccines = np.zeros(shape=(51, 6, 91))
solution = model.simulate(vaccinated=fake_vaccines)

In [25]:
solution.deceased[:,:,-1].sum()

137248.5320956391

In [10]:
predictions_df = load_and_clean_delphi_predictions(DELPHI_PREDICTIONS_PATH)

In [15]:
first = predictions_df[predictions_df.date == dt.datetime(2021, 1, 30)]['total_detected_deaths'].sum()

In [16]:
final = predictions_df[predictions_df.date == dt.datetime(2021, 4, 30)]['total_detected_deaths'].sum()

In [17]:
final - first

181068.12860906305